In [1]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 30)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(3)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]/div').click()

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)

i = 1
while (i <= real_cnt):
      scroll_down(driver) 
      i += 1

# Step 6. 전체 게시물의 원본 URL 추출하기
item=[]
count = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

all = soup.find('article','KC1QD').find_all('a')

for i in all:    
    url = i['href']
    item.append(url)  
    count += 1
    
    if count == cnt :
        break

# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

count = 0
for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    f = open(f_name, 'a',encoding='UTF-8')
        
    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')
    
    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)
            
            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다
   본 제품은 서진수가 교육용으로 특별 제작했으며 
   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다
   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^
1.인스타그램의 ID를 입력하세요: gachilabs
2.인스타그램의 비밀번호를 입력하세요: gachilabs123
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 강남맛집
4.크롤링 할 건수는 몇건입니까?: 30
5.파일이 저장될 경로만 쓰세요(기본경로 : c:\temp\ ) : C:/data/


요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/div/div/button"}
  (Session info: chrome=91.0.4472.124)


In [ ]:
#Chap 7-1. 이미지 다운로드용 웹크롤러 만들기
# Step 1. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import urllib
import time
import sys
import re
import math
import os
import random
import 

f_dir = input('사진을 저장할 폴더를 지정하세요(예: c:\data\) : ')
query_txt ='사진저장'

#Step 2. 파일을 저장할 폴더를 생성합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.chdir(f_dir)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
f_result_dir = f_dir+s+'-'+query_txt

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(path)

s_time = time.time( )      # 크롤링 시작 시간을 위한 타임 스탬프를 찍습니다

driver.get("https://korean.visitkorea.or.kr/detail/rem_detail.html?cotid=be3db10c-b642-409c-81cc-c4cdecb5bd8b&temp=")
time.sleep(2)  # 페이지가 모두 열릴 때 까지 2초 기다립니다.

# 학습목표 1: 자동 스크롤다운 함수 만들기
# Step 4.  스크롤다운 함수를 생성한 후 실행합니다.
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    
scroll_down(driver) 
    

# 본문의 사진 정보를 가져와서 지정된 폴더에 저장하기    
# Step 5. 이미지 추출하여 저장하기 

file_no = 0                                
count = 1
img_src2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img_src = soup.find('div','box_txtPhoto').find_all('img')

for i in img_src :
        img_src1=i['src']
        img_src2.append(img_src1)
        count += 1

for i in range(0,len(img_src2)) :
        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except :
                continue        
        file_no += 1                
        time.sleep(0.5)      
        print("%s 번째 이미지 저장중입니다=======" %file_no)
       
# Step 6. 요약 정보를 출력합니다                
e_time = time.time( )
t_time = e_time - s_time

store_cnt = file_no -1

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %f_result_dir)
print("=" *70)

driver.close( )

In [2]:
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd    
import os
import math

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 ")
print("=" *80)

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt/60)

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\\temp\\):")
if f_dir == '' :
    f_dir = "c:\\temp\\"
    
print("\n")

if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

#Step 3.저장될 파일 경로와 이름을 지정합니다
sec_name = '식품'
query_txt='쿠팡'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)
    
s_time = time.time( )

from bs4 import BeautifulSoup
import requests

url = 'https://www.coupang.com/'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')


#Step 4. 웹사이트 접속 후 해당 메뉴로 이동합니다.
chrome_path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
query_url='https://www.coupang.com/'
driver.get(query_url)
time.sleep(5)

# 분야별 더보기 버튼을 눌러 페이지를 엽니다
driver.find_element_by_xpath("""//*[@id="header"]/div""").click( )
driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[4]/a""").click( )

#Step 5. 내용을 수집합니다
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

ranking2=[]        #제품의 판매순위 저장
title2=[]          #제품 정보 저장
p_price2=[]        #현재 판매가 저장
discount2 = []     #할인율 저장
sat_count2=[]      #상품평 수 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)

scroll_down(driver)   #현재화면의 가장 아래로 스크롤다운합니다

for x in range(1,page_cnt + 1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    item_result = soup.find('ul','baby-product-list').find_all('li')

    for li in item_result :
        if cnt < count :
            break

        # 제품 이미지 다운로드 하기
        import urllib.request
        import urllib

        try :
            photo = li.find('dt','image').find('img')['src']
        except AttributeError :
            continue

        file_no += 1
        full_photo = 'https:' + photo
        urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
        time.sleep(0.5)

        #제품 내용 추출하기
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)

        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")

        try :
            t = li.find('div','name').get_text().replace("\n","")
        except :
            title = '제품소개가 없습니다'
            print(title.replace("\n",""))
            f.write('2.제품소개:'+ title + "\n")
        else :
            title = t.replace("\n","").strip()
            print("2.제품소개:", title.replace("\n","").strip())                  
            f.write('2.제품소개:'+ title + "\n")

        try :
            p_price = li.find('strong','price-value').get_text().replace("\n","")
        except :
            p_price = '0'
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")
        else :
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")

        try :
            discount = li.find('span','discount-percentage').get_text().replace("\n","")
        except  :
            discount = '0'
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")
        else :
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")

        try :
            sat_count_1 = li.find('span','rating-total-count').get_text()
            sat_count_2 = sat_count_1.replace("(","").replace(")","")
        except  :
            sat_count_2='0'
            print('5.상품평 수: ',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")
        else :
            print('5.상품평 수:',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")

        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        ranking2.append(ranking)
        title2.append(title.replace("\n",""))

        p_price2.append(p_price.replace("\n",""))
        discount2.append(discount)

        try :   
            sat_count2.append(sat_count_2)
        except IndexError :
            sat_count2.append(0)

        count += 1
    x += 1                
    driver.find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭
          
#step 6. csv , xls 형태로 저장하기              
co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['상품평수']=pd.Series(sat_count2)

# csv 형태로 저장하기
co_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
co_best_seller.to_excel(fx_name ,index=False)

e_time = time.time( )
t_time = e_time - s_time

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30   
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120   

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='B'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

driver.close( )

 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 
1.크롤링 할 건수는 몇건입니까?: ㅇㄴ


ValueError: invalid literal for int() with base 10: 'ㅇㄴ'

In [20]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.coupang.com/'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
print(soup)


<!DOCTYPE html>

<!--[if lte IE 7 ]><html class="lt-ie9 lt-ie8" lang="ko-KR"><![endif]-->
<!--[if IE 8 ]><html class="lt-ie9" lang="ko-KR"><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><html lang="ko-KR"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="zaNrGtrOLMjglkziY2IvmL8dOXyCWHGArDHqFazJQVI" name="google-site-verification"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//cart.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupang.com" rel="dns-prefetch"/>
<link href="//assets2.coupang.com" rel="dns-prefetch"/>
<link href="//assets.coupangcdn.com" rel="dns-prefetch"/>
<link href="//asset1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//private.coupang.com" rel="dns-prefetch"/>
<link href="//img1a.coupangcdn.com" rel="dns-prefetch"/>
<link href="//image1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//thumbnail1.coupangcdn.com" rel="dns-prefetch"/>
<link href="//static.coupangcdn.com" rel="dns-prefetch"/>
<link href="//www.facebook.

In [2]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/data/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[4]/div/a[1]/div/div/div[1]/span').click()

import pandas as pd

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)

print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]
    
a = 1
while (a <= real_cnt):
    scroll_down(driver) 

    # Step 6. 전체 게시물의 원본 URL 추출하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all = soup.find('article','KC1QD').find_all('a')

    for i in all:    
        url = i['href']
        item.append(url)          
        item2 = pd.Series(item).drop_duplicates()
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(item2) >= cnt :
        break
   
# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item2)) :
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > cnt:
        break

# 아래 for 반복문은 수집된 URL 주소를 확인하는 코드임        
#for c in range(1,len(full_url)+1) :
#    print(c,':',full_url[c-1])

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

count = 0

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    f = open(f_name, 'a',encoding='UTF-8')

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)

            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다
   본 제품은 서진수가 교육용으로 특별 제작했으며 
   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다
   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^
1.인스타그램의 ID를 입력하세요: gachilabs
2.인스타그램의 비밀번호를 입력하세요: gachilabs123
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 강남맛집
4.크롤링 할 건수는 몇건입니까?: 40
5.파일이 저장될 경로만 쓰세요(기본경로 : c:\temp\ ) : C:/data/


요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/div/div/button"}
  (Session info: chrome=91.0.4472.124)


In [2]:
# 유투브에서 검색 후 동영상의 리뷰 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("유투브 영상의 댓글을 수집하는 웹크롤러입니다")
print("=" *80)

query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir = 'c:\\temp\\'

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")   

 # Step 3. 저장될 파일위치와 이름을 지정 한 후 크롬 드라이버를 실행하여 페이지를 엽니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+'youtube'+'-'+s+'-'+query_txt)
os.chdir(f_dir+'youtube'+'-'+s+'-'+query_txt)

ff_dir=f_dir+'youtube'+'-'+s+'-'+query_txt
ff_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

# Step 2. 크롬 드라이버 설정 
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

#검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if reple_cnt < 20 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(cnt/20)

if cnt > 20 :
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
    item.append(i['href'])
    count += 1

    if count == cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == cnt :
        break

play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]

for y in range(0,len(full_url)) :
    driver.get(full_url[y])
    time.sleep(10)
    print("\n")
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    i = 1
    while ( i <=  page_cnt) :
        scroll_down(driver)
        time.sleep(5)
        i += 1

    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')

    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())

    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")

    t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()
    t_view_2 = t_view_1.replace("," ,"")
    t_view_3 = re.search("\d+",t_view_2)
    print(t_view_1)
    t_view_4 = int(t_view_3.group())
    
    print("=" *80)
    print("%s 번째 동영상의 조회수는 %s 회 이고 수집 가능한 댓글수는 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("\n")
    
    t_comwriter_2 = []
    t_comwriter_1 = soup2.select('#author-text > span')
    for com in range(len(t_comwriter_1)):
        t_comwriter_2.append(t_comwriter_1[com].get_text().strip())

    t_comdate_2 = []
    t_comdate_1 = soup2.select('#header-author > yt-formatted-string > a')
    for com in range(len(t_comdate_1)):
        t_comdate_2.append(t_comdate_1[com].get_text())
        
    t_comment_2 = []
    t_comment_1 = soup2.select('#content-text')
    for com in range(len(t_comment_1)):
        t_comment_2.append(t_comment_1[com].get_text())
        
   
    for rep in range(1, reple_cnt+1):
        
        print("%s 번째 영상의 %s 번째 댓글" %(play_cnt, str(rep)))
        print("-" *80)
        
        print("%s URL 주소: %s" %('1.', full_url[y]))
        print("%s 댓글 작성자명: %s" %('2.', t_comwriter_2[rep]))
        print("%s 댓글 작성일자: %s" %('3.', t_comdate_2[rep]))
        print("%s 댓글 내용: %s" %('4.', t_comment_2[rep]))
        print("\n")   
    
    print("=" *80)
    print("\n")

    play_cnt += 1

print("댓글 수집이 완료되었습니다")


유투브 영상의 댓글을 수집하는 웹크롤러입니다
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 올리브영
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:3
3.각 동영상에서 추출할 댓글은 몇건입니까?: 5
4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\temp\):C:/data/


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^


검색된 영상 중 3 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.
댓글 406개
1 번째 동영상의 조회수는 279678 회 이고 수집 가능한 댓글수는 총 406개 입니다
1 번째 동영상의 제목은 찐 애정템 보고 흥분을 감출 수 없었어!! 올리브영 탈탈 털어보았습니다�� 입니다


1 번째 영상의 1 번째 댓글
--------------------------------------------------------------------------------
1. URL 주소: https://www.youtube.com//watch?v=0q0kLn7_bHg
2. 댓글 작성자명: b bbb
3. 댓글 작성일자: 2주 전
4. 댓글 내용: 이 담날 정말 힘들엇서여...(올영 알바)


1 번째 영상의 2 번째 댓글
--------------------------------------------------------------------------------
1. URL 주소: https://www.youtube.com//watch?v=0q0kLn7_bHg
2. 댓글 작성자명: 김제로이종사촌
3. 댓글 작성일자: 2주 전
4. 댓글 내용: 트루님 입에서 좋다는 말이 나올때마다 검색 순위가 바뀌고 완판됐다는 전설의 라이브..


1 번째 영상의 3 번째 댓글
--------------------------------------------------------------------------------
1. URL 주소: https://www.youtube.com//

In [4]:
print(t_comment_1)

[<yt-formatted-string class="style-scope ytd-comment-renderer" id="content-text" slot="content" split-lines="">요즘 라네즈 네오 라인 영상이 자주 보여서 관심갖던 차에 이런 이벤트😍 후니언님이 피부에 쫜쫜하게 밀착된다고 그래서 써보고 싶었는데 이벤트 참여해봅니다❤</yt-formatted-string>, <yt-formatted-string class="style-scope ytd-comment-renderer" id="content-text" slot="content" split-lines="">마침 올영에서 쿠션 사려고 했는데 ㅠㅠㅠ 어떻게 이렇게 제 맘을 잘 아시나요 🥺 타이밍 굿💚</yt-formatted-string>, <yt-formatted-string class="style-scope ytd-comment-renderer" id="content-text" slot="content" split-lines="">맨날 쿠션 고민하는데 이거 넘 좋아보여용ㅋㅋㅋ써보고싶어요 ❤️‍🔥</yt-formatted-string>, <yt-formatted-string class="style-scope ytd-comment-renderer" id="content-text" slot="content" split-lines="">이쿠션 유튜브에서 진짜 많이 봤어요.  진짜 짱인 것 같더라구요.  같구 싶어여</yt-formatted-string>, <yt-formatted-string class="style-scope ytd-comment-renderer" id="content-text" slot="content" split-lines="">쿠션 유목민인데 요즘 이게 눈에 띄더라구요❤ 색도 제가 좋아하는 민트고 네모난 디자인이 너무 특색있고 이뻐요! 여름용 쿠션은 따로 없는데 한번 써보고싶어요❤</yt-formatted-string>, <yt-formatted-string class

In [5]:
t_comment_2

['요즘 라네즈 네오 라인 영상이 자주 보여서 관심갖던 차에 이런 이벤트😍 후니언님이 피부에 쫜쫜하게 밀착된다고 그래서 써보고 싶었는데 이벤트 참여해봅니다❤',
 '마침 올영에서 쿠션 사려고 했는데 ㅠㅠㅠ 어떻게 이렇게 제 맘을 잘 아시나요 🥺 타이밍 굿💚',
 '맨날 쿠션 고민하는데 이거 넘 좋아보여용ㅋㅋㅋ써보고싶어요 ❤️\u200d🔥',
 '이쿠션 유튜브에서 진짜 많이 봤어요.  진짜 짱인 것 같더라구요.  같구 싶어여',
 '쿠션 유목민인데 요즘 이게 눈에 띄더라구요❤ 색도 제가 좋아하는 민트고 네모난 디자인이 너무 특색있고 이뻐요! 여름용 쿠션은 따로 없는데 한번 써보고싶어요❤',
 '일단 비주얼부터 동글매끈 엄청 귀여워요💚 여름에 쓰기 좋을 것 같아요!!',
 '저 모양이 다른데서는 없는 네모라서 특이하고 무튼 너무 이뻐요 ㅎㄹ',
 '리마가 리뷰해 주었으면 좋겠다고 생각하는 상품은 "구강청결제"입니다.\n코로나19의 등장으로, 실내/실외 공간에서 하루 종일 마스크를 착용하다보니 자신의 입냄새를 느껴버렸습니다.\n어떤 제품을 선택해야 좋을 지 계속 고민하고 있었는데, 이에 대한 부분을 다루어 주시면 정말 좋을 것 같습니다.\n합리적인 소비를 할 수 있도록 도와주세요!!!',
 '라네즈 네오쿠션 좋다고 많이 들어서 어떨지 궁금했는데 리뷰 너무 좋아요♥️👍',
 '모델분 마마무 솔라 닮으셨어요 !!',
 '네오쿠션 여름에 땀이 많이 날때 사용하는데 너무 뽀송 하고 좋아요♥',
 '홍현희님이 나오신 귀여운 광고 영상보고  궁금💭 했는데친절한 설명 감사합니다 ☺️',
 '맨날 쿠션 영상만 보면서 쿠션은 안사는 사람 바로 나....ㅋㅋㅋㅋㅋㅌ',
 '이번에 네오 파운데이션이 나와서 사봤는데 너무 좋아서 쿠션도 한번 써보고싶어요! 네오 쿠션 좋다고 주변에서 자주 말해서 이벤트 참여 해봅니다',
 '매트쿠션은 피부에서 뜨는 느낌이라 잘 안쓰는데 네오쿠션 좋다는 얘기를 너무 많이 들어서 궁금해요!',
 '라네즈 쿠션 좋다고 많이 들었는데 한번 사용해보고싶네용!!! 

In [ ]:
import pandas as pd



co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['상품평수']=pd.Series(sat_count2)

In [2]:
# 연습문제 7: 쿠팡 분야별 베스트 상품 크롤러
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import urllib.request
import urllib

print("=" *80)
print("  연습문제 : 쿠팡 사이트의 베스트셀러 상품 정보 수집하기 ")
print("=" *80)

query_txt='쿠팡'
query_url='https://www.coupang.com/'

sec = input('''
 1.패션의류/잡화-남성    2.패션의류/잡화-여성   3.뷰티              4.출산/유아동
 5.식품                  6.주방용품             7.생활용품          8.홈인테리어
 9.가전/디지털          10.스포츠/레저         11.자동차용품       12.도서/음반/DVD
13.완구/취미            14.문구/오피스         15.반려동물용품     16.헬스/건강식품


1.위 분야 중에서 자료를 수집할 분야의 번호를  선택하세요: ''')

cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt/60)

f_dir = input('3.파일을 저장할 폴더명만 쓰세요(엔터키 입력시 :c:\\temp\\ 아래에 저장됨):')
if f_dir=='' :
    f_dir="c:\\temp\\"

print("\n")

if sec == '1' :
  sec_name='패션의류_잡화-남성'
elif sec =='2' :
  sec_name='패션의류_잡화-여성'
elif sec =='3' :
  sec_name='뷰티'
elif sec =='4' :
  sec_name='출산_유아동'
elif sec =='5' :
  sec_name='식품'
elif sec =='6' :
  sec_name='주방용품'
elif sec =='7' :
  sec_name='생활용품'
elif sec =='8' :
  sec_name='홈인테리어'
elif sec =='9' :
  sec_name='가전_디지털'
elif sec =='10' :
  sec_name='스포츠_레저'
elif sec =='11' :
  sec_name='자동차용품'
elif sec =='12' :
  sec_name='도서_음반_DVD'
elif sec =='13' :
  sec_name='완구_취미'
elif sec =='14' :
  sec_name='문구_오피스'
elif sec =='15' :
  sec_name='반려동물용품'
elif sec =='16' :
  sec_name='헬스_건강식품'


if cnt > 30 :
  print("    요청 건수가 많아서 시간이 약간 소요되오니 잠시만 기다려 주세요~~")
else :
  print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

s_time = time.time( )

  연습문제 : 쿠팡 사이트의 베스트셀러 상품 정보 수집하기 

 1.패션의류/잡화-남성    2.패션의류/잡화-여성   3.뷰티              4.출산/유아동
 5.식품                  6.주방용품             7.생활용품          8.홈인테리어
 9.가전/디지털          10.스포츠/레저         11.자동차용품       12.도서/음반/DVD
13.완구/취미            14.문구/오피스         15.반려동물용품     16.헬스/건강식품


1.위 분야 중에서 자료를 수집할 분야의 번호를  선택하세요: 5
2.크롤링 할 건수는 몇건입니까?: 10
3.파일을 저장할 폴더명만 쓰세요(엔터키 입력시 :c:\temp\ 아래에 저장됨):C:/data/


    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~


In [3]:
#Step 3. 검색어 입력한 후 검색하여 해당 장르로 이동하기
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

query_url='https://www.coupang.com/'

driver.get(query_url)
time.sleep(3)

# Access Denied 메시지가 나오면 아래코드로 쿠키를 삭제한다
driver.delete_all_cookies()
time.sleep(2)


# 분야별 더보기 버튼을 눌러 페이지를 엽니다
driver.find_element_by_xpath("""//*[@id="header"]/div""").click( )

if sec == '1' :   # 남성패션
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[1]/a""").click( )
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[1]/div/div/ul/li[2]/a""").click()
elif sec == '2' :  #여성패션
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[1]/a""").click( )
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[1]/div/div/ul/li[1]/a""").click()
elif sec == '3' :   #뷰티
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[2]/a""").click( )
elif sec == '4' :    #출산_유아동
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[3]/a""").click( )
elif sec == '5' :    #식품
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[4]/a""").click( )
elif sec == '6' :    #주방용품
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[5]/a""").click( )
elif sec == '7' :    #생활용품
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[6]/a""").click( )
elif sec == '8' :    #홈인테리어
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[7]/a""").click( )
elif sec == '9' :    #가전디지털
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[8]/a""").click( )
elif sec == '10' :   #스포츠레저
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[9]/a""").click( )
elif sec == '11' :   #자동차용품
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[10]/a""").click( )
elif sec == '12' :   #도서음반DVD
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[11]/a""").click( )
elif sec == '13' :   # 완구취미
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[12]/a""").click( )
elif sec == '14' :  #문구오피스
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[13]/a""").click( )
elif sec == '15' :   #반려동물
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[14]/a""").click( )
elif sec == '16' :   #헬스건강
  driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[15]/a""").click( )

#Step 5. 화면을 스크롤해서 아래로 이동합니다

def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(1)

scroll_down(driver)

ranking2=[]
title2=[]
o_price2=[]
p_price2=[]
discount2 = []
rocket2=[]
soldout2=[]
score2=[]
sat_count2=[]

count = 1
file_no = 1

#Step 6. 내용을 수집합니다
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

time.sleep(2)

for x in range(1,page_cnt + 1) :
    
    scroll_down(driver)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #이미지 추출하기
    img_src2=[]   # 이미지 URL 저장변수
    
    item_result = soup.find('ul','baby-product-list')
    item_result2 = item_result.find_all('li')

    for li in item_result2:

            if count > cnt :
                break

            try :
                photo = li.find('dt','image').find('img')['src']
            except AttributeError :
                continue
                
            full_photo = 'https:' + photo
            urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
            time.sleep(0.5)

            if  file_no > cnt :
                break

            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")
            print("-" *70)
            
            ranking = count
            print("1.판매순위:",ranking)
            f.write('1.판매순위:'+ str(ranking) + "\n")

            try :
                t = li.find('div',class_='name').get_text().replace("\n","")
            except  :
                title = '제품소개가 없습니다'
                print(title.replace("\n",""))
                f.write('2.제품소개:'+ title + "\n")
            else :
                title = t.replace("\n","").strip()
                print("2.제품소개:", title.replace("\n","").strip())                  
                f.write('2.제품소개:'+ title + "\n")

            try :
                p_price = li.find('strong','price-value').get_text().replace("\n","")
            except  :
                p_price = '0'
                print("3.판매가격:", p_price.replace("\n",""))
                f.write('3.판매가격:'+ p_price + "\n")
            else :
                print("3.판매가격:", p_price.replace("\n",""))
                f.write('3.판매가격:'+ p_price + "\n")

            try :
                o_price = li.find('del','base-price').get_text().replace("\n","")
            except  :
                o_price = p_price
                print("4.원래가격:",o_price.replace("\n",""))
                f.write('4.원래가격:'+ o_price + "\n")
            else :
                print("4.원래가격:",o_price.replace("\n",""))
                f.write('4.원래가격:'+ o_price + "\n")

            try :
                discount = li.find('span','discount-percentage').get_text().replace("\n","")
            except  :
                discount = '0'
                print("5:할인률:", discount)
                f.write('5.할인율:'+ discount + "\n")
            else :
                print("5:할인률:", discount)
                f.write('5.할인율:'+ discount + "\n")

            try :
                rocket = li.find('span','badge rocket').find('img')['alt']
            except  :
                rocket = ''
                print("6:로켓배송여부:", rocket)
                f.write('6:로켓배송여부:'+ rocket + "\n")
            else :
                print("6:로켓배송여부:", rocket)
                f.write('6:로켓배송여부:'+ rocket + "\n")

            try :
                soldout = li.find('div','out-of-stock').get_text().replace("\n","").strip()
            except :
                soldout = ''
                print("7:품절여부:", soldout)
                f.write('7.품절여부:'+ soldout + "\n")
            else :
                print("7:품절여부:", soldout)
                f.write('7.품절여부:'+ soldout + "\n")

            try :
                sat_count = li.find('span','rating-total-count').get_text().replace("(","").replace(")","")
            except  :
                sat_count='0'
                print('8.상품평수: ',sat_count)
                f.write('8.상품평수:'+ sat_count + "\n")
            else :
                print('8.상품평수:',sat_count)
                f.write('8.상품평수:'+ sat_count + "\n")

            try :
                score= li.find('em','rating').get_text()
            except  :
                score='0'
                print('9.평점: 5.0 /',score)
                f.write('9.평점: 5.0 /'+ score + "\n")
            else :     
                print('9.평점: 5.0 /',score)
                f.write('9.평점: 5.0 /'+ score + "\n")

            print("-" *70)

            f.close( )

            time.sleep(0.6)

            ranking2.append(ranking)
            title2.append(title.replace("\n",""))
            o_price2.append(o_price.replace("\n",""))
            p_price2.append(p_price.replace("\n",""))
            rocket2.append(rocket.replace("\n",""))
            soldout2.append(soldout)
            discount2.append(discount)
            score2.append(score)

            try :   
                sat_count2.append(sat_count)
            except IndexError :
                sat_count2.append(0)
                
            count += 1
            file_no += 1
            
    if cnt:
        x += 1     

        driver.delete_all_cookies()
        time.sleep(2)

        driver.find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭


#step 9. 엑셀 형태로 저장하기

co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품정가']=pd.Series(o_price2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['로켓배송여부']=pd.Series(rocket2)
co_best_seller['상품평수']=pd.Series(sat_count2)
co_best_seller['상품평점']=pd.Series(score2)
co_best_seller['품절여부']=pd.Series(soldout2)


# csv 형태로 저장하기
co_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
co_best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

#driver.close( )

count -= 1

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
print(" 잠시만 기다리시면 수집된 결과가 저장된 엑셀 파일이 자동으로 열립니다~~^^")
print("=" *80)

#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)
    time.sleep(0.3)

for b in range(1,cnt+1) :
    file_name=img_dir+'/'+str(b)+'.jpg'
    file_name2.append(file_name)
    time.sleep(0.3)

for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    time.sleep(0.3)



===== 곧 수집된 결과를 출력합니다 ^^ ===== 


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 폴바셋 파나마 게이샤 프라이빗 컬렉션 원두+리유저블 텀블러 증정, 100g
3.판매가격: 23,750
4.원래가격: 25,000
5:할인률: 5%
6:로켓배송여부: 로켓배송
7:품절여부: 
8.상품평수: 13
9.평점: 5.0 / 4.0
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: 엠수산 국내산 바로손질한 고창장어 1~2미 + 순한맛 소스 50g + 매운맛 소스 50g + 락교와초생강 60g, 650g, 1개
3.판매가격: 35,900
4.원래가격: 39,900
5:할인률: 10%
6:로켓배송여부: 로켓배송
7:품절여부: 
8.상품평수: 9
9.평점: 5.0 / 5.0
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 제주 삼다수, 2L, 24개
3.판매가격: 23,500
4.원래가격: 39,600
5:할인률: 40%
6:로켓배송여부: 로켓배송
7:품절여부: 
8.상품평수: 306221
9.평점: 5.0 / 5.0
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"2"}
  (Session info: chrome=91.0.4472.124)
